In [714]:
from typing import Generic,TypeVar,Protocol
import sqlite3
import copy
import pandas as pd
import random

T = TypeVar('T')

DATATYPES = {
    'int':'INTEGER',
    'str':'TEXT',
    'float':'REAL',
    'bytes':'BLOB'
}

class Field(Generic[T]):

    def __init__(
        self,
        value:T=None,
        /,
        init=True,
        repr=True,
        hash=None,
        compare=True,
        metadata=dict(),
        kw_only=False,
        primary_key:bool = False,
        required:bool = False,
        unique:bool = False,
    ) -> None:
        self.__allow_init__ = init
        self.__value__:T = value
        self.__type__:type[T] = type(value)
        self.__sql_primary_key = primary_key
        self.__sql_required = required
        self.__sql_unique = unique
        self.__query__ = ''
        self.__sql__ = f'{DATATYPES[self.__type__.__name__]} {self.__constraint()}'
        
    def __constraint(self) -> str:

        constraints = []

        if self.__sql_primary_key:
            constraints.append("PRIMARY KEY")

        if self.__sql_unique:
            constraints.append("UNIQUE")

        if self.__sql_required:
            constraints.append("NOT NULL")

        return ' '.join(constraints)

    def __repr__(self) -> str:
        return str(self.__value__)
    
    def __str__(self) -> str:
        return str(self.__value__)
    
    def __eq__(self, __value: object):

        if str(__value).isnumeric():
            self.__query__ = "{query}={value}".format(table='table',query=self.__query__,value=__value)
        else:
            self.__query__ = "{query} LIKE '%{value}%'".format(table='table',query=self.__query__,value=__value)
        self.__value__ = self.__value__ == __value
        return self
    
    def __gt__(self, __value: object):

        if isinstance(__value,Field):
            __value = str(__value)

        self.__value__ = self.__value__ > __value
        self.__query__ = "{table}.{query} > {value}".format(table='table',query=self.__query__,value=__value)
        return self
    
    def __lt__(self, __value: object):

        if isinstance(__value,Field):
            __value = str(__value)

        self.__value__ = self.__value__ == __value
        self.__query__ = "{table}.{query} < {value}".format(table='table',query=self.__query__,value=__value)
        return self
    
    def __ge__(self, __value: object):

        if isinstance(__value,Field):
            __value = str(__value)

        self.__value__ = self.__value__ == __value
        self.__query__ = "{table}.{query} >= {value}".format(table='table',query=self.__query__,value=__value)
        return self
    
    def __le__(self, __value: object):

        if isinstance(__value,Field):
            __value = str(__value)

        self.__value__ = self.__value__ <= __value
        self.__query__ = "{table}.{query} <= {value}".format(table='table',query=self.__query__,value=__value)
        return self
    
    def __ne__(self, __value: object):

        if isinstance(__value,Field):
            __value = str(__value)

        self.__value__ = self.__value__ != __value
        self.__query__ = "{table}.{query} <> {value}".format(table='table',query=self.__query__,value=__value)
        return self
    
    def set(self,value:T) -> bool:
        assert isinstance(value,self.__type__),'Invalid datatype'
        self.__value__ = value
        return True    
    def get(self) -> T:
        return self.__value__


In [715]:
class BaseModel(Generic[T]):

    def __init__(self,**kwargs:T) -> None:

        kwargs = kwargs.copy()

        self.__sql_table_name__ = self.__class__.__name__.lower()
        self.__name__ = self.__class__.__name__
        self.__query__ = ''
        self.__sql__ = self.__createTable()
        self.__table_info__ = {}
        if len(kwargs) == 0:
            return
        
        for attr in self.__class__.__annotations__.keys():
            field_col:Field = copy.deepcopy(getattr(self,attr))
            field_col.__query__ = attr
            if len(kwargs) == 0:
                field_col.set(None)
                setattr(self,attr,field_col)
                self.__table_info__.update({attr:getattr(self,attr)})
            if not field_col.__allow_init__:
                continue
            assert attr in kwargs.keys(),f"Field '{attr}' required."
            field_col.set(kwargs[attr])
            setattr(self,attr,field_col)
            self.__table_info__.update({attr:getattr(self,attr)})

    def __createTable(self) -> str:

        columns:list[Field] = []

        for key in self.__annotations__.keys():
            field_info:Field = getattr(self,key)
            columns.append(f"{key} {field_info.__sql__}")
        
        return f"CREATE TABLE {self.__sql_table_name__} ( {', '.join(columns)} );"
    
    @staticmethod
    def __execute_table__(engine:sqlite3.Connection) -> None:

        tables:list[BaseModel] = BaseModel.__subclasses__()

        sql_table_scripts:list[str] = []

        for table in tables:
            sql_table_scripts.append(table().__sql__)

        sql_statement = f"BEGIN; {' '.join(sql_table_scripts)} COMMIT;"

        cursor = engine.cursor()

        cursor.executescript(sql_statement)

        cursor.close()

    def __repr__(self) -> str:
        return f"{self.__sql_table_name__}({', '.join([f'{key}={val}' for key,val in self.__table_info__.items()])})"

class SampleUser(BaseModel):
    id:Field[int] = Field(int(),primary_key=True)
    name:Field[str] = Field(str(),required=True)
    email:Field[str] = Field(str(),required=True,unique=True)
    phone_number:Field[int] = Field(int(),required=True,unique=True)

class SampleItem(BaseModel):
    id:Field[int] = Field(int(),primary_key=True)
    name:Field[str] = Field(str(),required=True)
    price:Field[float] = Field(float(),required=True)
    contact_info:Field[int] = Field(int(),required=True)


In [716]:
class Query(Generic[T]):

    def __init__(self,table:T) -> None:
        self.table:BaseModel[T] = table
        self.__query__ = []
        self.__values__ = ''
        self.__table_object__ = globals()[table.__name__]
        self.__sql__ = f'BEGIN; {self.__query__} COMMIT;'

    def select(self,*columns:tuple[T]):
        
        sql_select_syntax = "SELECT {columns} FROM {table}"

        if len(columns) == 0:

            self.__query__.append(
                sql_select_syntax.format(
                    columns=', '.join([
                        x.__query__ for x in self.table.__table_info__.values()
                    ]),
                    table=self.table.__sql_table_name__
                )
            )

            return self

        self.__query__.append(
            sql_select_syntax.format(
                columns=', '.join([
                    x.__query__ for x in columns
                ]),
                table=self.table.__sql_table_name__
            )
        )
        return self
    
    def insert(self,*values:tuple[T]|T):

        sql_insert_syntax = "INSERT INTO {table} ( {columns} ) VALUES ( {values} )"

        if len(values) == 1 and isinstance(values[0],BaseModel):
            self.__values__ = [val.__value__ for val in values[0].__table_info__.values()]

            self.__query__.append(
                sql_insert_syntax.format(
                    table=self.table.__sql_table_name__,
                    columns=', '.join([x.__query__ for x in values[0].__table_info__.values()]),
                    values=', '.join(self.__fillValue(len(self.__values__)))
                )
            )
            return self

        self.__values__ = tuple([x.__value__ for x in values])

        self.__query__.append(
            sql_insert_syntax.format(
                table=self.table.__sql_table_name__,
                columns=', '.join([x.__query__ for x in values]),
                values=', '.join(self.__fillValue(len(self.__values__)))
            )
        )
        return self

    def where(self,conditions:T):

        sql_where_syntax = "WHERE {conditions}"

        self.__query__.append(
            sql_where_syntax.format(
                conditions=conditions.__query__
            )
        )

        return self

    def __fillValue(self,length:int) -> list[str]:

        return ['?' for _ in range(length)]
    
    def __repr__(self) -> str:
        return ' '.join(self.__query__)
    
    def __str__(self) -> str:
        return ' '.join(self.__query__)
        
test_val = SampleUser(id=1,name='karl',email='karl@gmail.com',phone_number=639055557351)
test_val2 = SampleUser(id=2,name='karl robeck alferez',email='vienne.va0922@gmail.com',phone_number=6393899123)

In [717]:
class Session(Generic[T]):

    def __init__(self,engine:sqlite3.Connection) -> None:
        self.cursor = engine.cursor()
        self.connection = engine
        self.__query_obj__ = None
        pass

    def __enter__(self):
        return self
    
    def __exit__(self,exc_type, exc_value, exc_traceback):
        if exc_type:
            print(exc_type)
        if exc_value:
            print(exc_value)
        if exc_traceback:
            print(exc_traceback)
        self.cursor.close()
        return self
    
    def execute(self,sql:Query[T]):
        sql.__values__ = [f"'{item}'" if isinstance(item, str) else item for item in sql.__values__]
        self.cursor = self.cursor.execute(str(sql),sql.__values__)
        self.__query_obj__ = sql.__table_object__
        return self
    
    def fetchall(self,to:str='base_model'):
        columns = tuple([x[0] for x in self.cursor.description])
        data = self.cursor.fetchall()
        if to == 'base_model':
            return self.__query_obj__(**dict(zip(columns,data[0])))
        
        if to == 'pandas':
            if len(data) == 1:
                return pd.DataFrame(data=[data[0]],columns=columns)
            return pd.DataFrame(data=data,columns=columns)
    
    def commit(self):
        return self.connection.commit()
    


In [718]:
conn = sqlite3.connect(':memory:')
BaseModel.__execute_table__(conn)


In [719]:
with Session(conn) as db:
    data = SampleUser(id=0,name=f'karl 1',email=f'karl_@gmail.com',phone_number=random.randint(1,1000000))
    select_stmt = Query(data).select().where(data.email == "karl_32@gmail.com")
    
    for _ in range(100):

        sample_data = SampleUser(id=_,name=f'karl {_} 1',email=f'karl_{_}@gmail.com',phone_number=random.randint(1,1000000))

        insert_smth = Query(sample_data).insert(sample_data)
        db.execute(insert_smth)
        db.commit()
    print(select_stmt)
    print(db.execute(select_stmt).fetchall(to='pandas'))
cursor = conn.cursor()

SELECT id, name, email, phone_number FROM sampleuser WHERE email LIKE '%karl_32@gmail.com%'
   id         name                email  phone_number
0  32  'karl 32 1'  'karl_32@gmail.com'        750710
